# Getting Access Token and Setting Up HTTPS Request Headers

In [1]:
import requests as rs
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
from azure.storage.blob import ContainerClient,BlobServiceClient
from io import StringIO
import numpy as np
import calendar
import json
import datetime as dt
import warnings
import time


In [ ]:

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)

secrets = json.load(open(file='./secrets.json'))

refresh_token = secrets['refresh_token']

app_id = secrets['app_id']

app_secret = secrets['app_secret']

request_body = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token,
        "scope": "openid",
        "resource": "https://api.partnercenter.microsoft.com",
        "client_id": app_id,
        "client_secret": app_secret,
    }

base_url = (
'https://api.partnercenter.microsoft.com'
)

In [2]:
def getAccessToken(request_body):
    req = rs.post(
                  "https://login.windows.net/6e75cca6-47f0-47a3-a928-9d5315750bd9/oauth2/token"
                  ,data = request_body)
    access_token = json.loads(req.text)['access_token']
    return access_token



In [3]:
access_token = getAccessToken(request_body = request_body)

headers = {'Authorization': 'Bearer ' + access_token}

# Get Invoices

In [4]:
def getInvoices (base_url):
    relative_url = '/v1/invoices'

    response = json.loads(
    rs.get(
            f"{base_url}{relative_url}",
           headers=headers)
    .content)

    main_df=pd.DataFrame() # Main DataFrame, currently empty

    columns = list(response['items'][0].keys())

    for row in response['items']:
        row_df = pd.DataFrame.from_dict([row]) #Each row gets converted to a DataFrame
        main_df = pd.concat([main_df,row_df],ignore_index=True) #Then gets appended to the Main DataFrame
        
    return main_df

In [5]:
invoices_df = getInvoices(base_url = base_url)
invoices_df = invoices_df[invoices_df['invoiceType']=='OneTime']


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
invoices_df

In [35]:
invoiceIdList = list(invoices_df['id'])
invoiceDateList = list(invoices_df['invoiceDate'])

zipp = zip(invoiceIdList,invoiceDateList)
# latestInvoice,latestInvoiceDate = list(zipp)[-1]   


In [36]:
list(zipp)[-1]

('G021249609', '2023-04-06T03:00:28.1147298Z')

# Get Invoice Line Items


In [11]:
def getInvoiceLineItems(base_url, invoiceId):
    relative_url = f'/v1/invoices/{invoiceId}/lineitems/OneTime/BillingLineItems?size=5000'

    response = json.loads(
        rs.get(
                f"{base_url}{relative_url}",
               headers=headers)
        .content)

    main_df=pd.DataFrame()

    columns = list(response['items'][0].keys())

    for row in response['items']:
        sub_df = pd.DataFrame.from_dict([row])
        main_df = main_df.append(sub_df,ignore_index=True)
    return main_df
    

In [12]:
invoice_df = getInvoiceLineItems(base_url = base_url, invoiceId = 'G020077203')
invoice_df.head()

,partnerId,customerId,customerName,customerDomainName,customerCountry,invoiceNumber,mpnId,resellerMpnId,orderId,orderDate,productId,skuId,availabilityId,productName,skuName,productQualifiers,chargeType,unitPrice,effectiveUnitPrice,unitType,quantity,subtotal,taxTotal,totalForCustomer,currency,publisherName,publisherId,subscriptionDescription,subscriptionId,subscriptionStartDate,subscriptionEndDate,chargeStartDate,chargeEndDate,termAndBillingCycle,alternateId,referenceId,priceAdjustmentDescription,discountDetails,pricingCurrency,pcToBCExchangeRate,pcToBCExchangeRateDate,billableQuantity,meterDescription,billingFrequency,reservationOrderId,invoiceLineItemType,billingProvider,promotionId,attributes
0,6e75cca6-47f0-47a3-a928-9d5315750bd9,c5394d23-4adf-4ef3-b5b1-8aba00661d70,Semnox Solutions Private Limited,semnox.com,IN,G020077203,2240443,0,1659ce72905c,2023-02-16T09:32:15.6118422Z,CFQ7TTC0LH18,0001,CFQ7TTC0QVDL,Microsoft 365 New Commerce Promotional Offer A...,Microsoft 365 Business Basic,[],addQuantity,100.0,-15.32,,170,-2604.40,-469.20,-3073.60,INR,Microsoft Corporation,,,a1d18b55-6128-40c5-cf54-0b0971e397c8,2022-05-19T00:00:00Z,2023-03-20T00:00:00Z,2023-02-16T09:32:15.6118422Z,2023-02-20T00:00:00Z,One-Year commitment for monthly/yearly billing,1659ce72905c,ca6903e5-27a2-425b-830e-10fffbb7a0e9,,,INR,1.0,2023-02-01T00:00:00,170.0,,Monthly,a1d18b55-6128-40c5-cf54-0b0971e397c8,billing_line_items,one_time,39NFJQT1W0JK:0003:39NFJQT1Q5L4,{'objectType': 'OneTimeInvoiceLineItem'}
1,6e75cca6-47f0-47a3-a928-9d5315750bd9,93698627-0574-46ba-9592-ca538bad4e02,1to1help.net,1to1help.net,IN,G020077203,2240443,0,911c8d110a5c,2023-02-01T14:06:11.0116288Z,CFQ7TTC0LH18,0001,CFQ7TTC0PXL6,Microsoft 365 Business Basic,Microsoft 365 Business Basic,[],addQuantity,100.0,-25.81,,240,-6194.40,-1116.00,-7310.40,INR,Microsoft Corporation,,,33e313ea-acb1-4a37-dd0c-f01b9136437d,2022-09-09T00:00:00Z,2023-09-08T00:00:00Z,2023-02-01T14:06:11.0116288Z,2023-02-08T00:00:00Z,One-Year commitment for monthly/yearly billing,911c8d110a5c,a75fcacc-c134-4301-8c6c-5173e15c007f,,,INR,1.0,2023-02-01T00:00:00,240.0,,Monthly,33e313ea-acb1-4a37-dd0c-f01b9136437d,billing_line_items,one_time,,{'objectType': 'OneTimeInvoiceLineItem'}
2,6e75cca6-47f0-47a3-a928-9d5315750bd9,317bb271-825c-412a-b618-e76c7f845528,Global Tree Careers Private Limited,globaltree.co.in,IN,G020077203,2240443,0,ffa44db7de8d,2023-02-06T11:46:52.6129565Z,CFQ7TTC0LH18,0001,CFQ7TTC0QHPZ,Microsoft 365 Business Basic,Microsoft 365 Business Basic,[],addQuantity,100.0,-3.23,,235,-759.05,-136.30,-895.35,INR,Microsoft Corporation,,,8b22d290-ee75-44ba-da19-c6c6748136a1,2022-06-07T00:00:00Z,2023-06-06T00:00:00Z,2023-02-06T11:46:52.6129565Z,2023-02-06T00:00:00Z,One-Year commitment for monthly/yearly billing,ffa44db7de8d,f283274a-a8ed-4bf3-91c6-79501182b309,,,INR,1.0,2023-02-01T00:00:00,235.0,,Monthly,8b22d290-ee75-44ba-da19-c6c6748136a1,billing_line_items,one_time,,{'objectType': 'OneTimeInvoiceLineItem'}
3,6e75cca6-47f0-47a3-a928-9d5315750bd9,c751444d-b7e3-4837-9cc1-e267cac4a32a,Ticvic Technologies Pvt Ltd,ticvic.com,IN,G020077203,2240443,0,f04031e541e6,2023-02-01T04:42:54.3534662Z,CFQ7TTC0LH18,0001,CFQ7TTC0R999,Microsoft 365 Business Basic,Microsoft 365 Business Basic,[],addQuantity,100.0,-48.39,,66,-3193.74,-574.86,-3768.60,INR,Microsoft Corporation,,,9533e044-cf54-419b-daee-8330300461f9,2023-01-19T00:00:00Z,2024-01-15T00:00:00Z,2023-02-01T04:42:54.3534662Z,2023-02-15T00:00:00Z,One-Year commitment for monthly/yearly billing,f04031e541e6,629a681a-ec81-45d4-a2a1-b360c13b5fd8,,,INR,1.0,2023-02-01T00:00:00,66.0,,Monthly,9533e044-cf54-419b-daee-8330300461f9,billing_line_items,one_time,,{'objectType': 'OneTimeInvoiceLineItem'}
4,6e75cca6-47f0-47a3-a928-9d5315750bd9,4ab0b6b8-cf39-4c1a-a2e6-29f851f0f8dd,Mobilution IT Systems Pvt Ltd,mobilutionit.com,IN,G020077203,2240443,0,29ad44e0ae50,2023-02-03T06:35:33.8349132Z,CFQ7TTC0LH1G,0001,CFQ7TTC0JPMV,Microsoft 365 Apps for business,Microsoft 365 Apps for business,[],addQuantity,432.

In [ ]:
invoice_df[
    (invoice_df['customerName'].str.startswith('TATA'))
          ][
    ['productName','skuName','termAndBillingCycle','totalForCustomer','chargeStartDate','chargeEndDate']
        ].sort_values('productName')
    